In [45]:
import io
import requests
import pandas as pd
import numpy as np

# create the dataframe reading the given input file

In [46]:
dfs = pd.read_html('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M',header =0, flavor='bs4', )[0]
dfs1 = dfs[['Postcode', 'Borough', 'Neighbourhood']]
dfs1.head(3)

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods


# Only process the cells that have an assigned borough. Ignore cells with a borough that is Not assigned

In [47]:
dfs2 = dfs1[dfs1['Borough']!= 'Not assigned']
dfs2.head(10)

,Postcode,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M5A,Downtown Toronto,Regent Park
6,M6A,North York,Lawrence Heights
7,M6A,North York,Lawrence Manor
8,M7A,Queen's Park,Not assigned
10,M9A,Etobicoke,Islington Avenue
11,M1B,Scarborough,Rouge
12,M1B,Scarborough,Malvern


# If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough. So for the 9th cell in the table on the Wikipedia page, the value of the Borough and the Neighborhood columns will be Queen's Park

In [48]:
dfs3 = dfs2[dfs2['Neighbourhood']== 'Not assigned']
dfs3

,Postcode,Borough,Neighbourhood
8,M7A,Queen's Park,Not assigned


# More than one neighborhood can exist in one postal code area. For example, in the table on the Wikipedia page, you will notice that M5A is listed twice and has two neighborhoods: Harbourfront and Regent Park. These two rows will be combined into one row with the neighborhoods separated with a comma as shown in row 11 in the above table

In [49]:
dfs2['UpdatedNeighbourhood'] = np.where(dfs2.Neighbourhood == 'Not assigned',dfs2.Borough,dfs2.Neighbourhood)
dfs2.head(10)

/opt/conda/envs/Python36/lib/python3.6/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


,Postcode,Borough,Neighbourhood,UpdatedNeighbourhood
2,M3A,North York,Parkwoods,Parkwoods
3,M4A,North York,Victoria Village,Victoria Village
4,M5A,Downtown Toronto,Harbourfront,Harbourfront
5,M5A,Downtown Toronto,Regent Park,Regent Park
6,M6A,North York,Lawrence Heights,Lawrence Heights
7,M6A,North York,Lawrence Manor,Lawrence Manor
8,M7A,Queen's Park,Not assigned,Queen's Park
10,M9A,Etobicoke,Islington Avenue,Islington Avenue
11,M1B,Scarborough,Rouge,Rouge
12,M1B,Scarborough,Malvern,Malvern


In [50]:
group_data = dfs2.groupby(['Postcode','Borough'])['UpdatedNeighbourhood'].agg([('UpdatedNeighbourhood', ', '.join)])
print (group_data)


                                                        UpdatedNeighbourhood
Postcode Borough                                                            
M1B      Scarborough                                          Rouge, Malvern
M1C      Scarborough                  Highland Creek, Rouge Hill, Port Union
M1E      Scarborough                       Guildwood, Morningside, West Hill
M1G      Scarborough                                                  Woburn
M1H      Scarborough                                               Cedarbrae
M1J      Scarborough                                     Scarborough Village
M1K      Scarborough             East Birchmount Park, Ionview, Kennedy Park
M1L      Scarborough                         Clairlea, Golden Mile, Oakridge
M1M      Scarborough         Cliffcrest, Cliffside, Scarborough Village West
M1N      Scarborough                             Birch Cliff, Cliffside West
M1P      Scarborough       Dorset Park, Scarborough Town Centre, Wexford ...

# In the last cell of your notebook, use the .shape method to print the number of rows of your dataframe

In [51]:
group_data.shape

(103, 1)